# Training Data

### Labeling DataSet

In [45]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import load_img
from sklearn.metrics import classification_report 
import numpy as np
import sklearn
import os 
import shutil 
import random

In [46]:
train_gen = ImageDataGenerator(rescale=1./255)

# Turning every folder name into class label
train_data = train_gen.flow_from_directory(
    "../Datasets/combined-cleaned-dataset",
    target_size=(224,224),
    batch_size=32,
    class_mode= "categorical"
)

print(train_data.class_indices)

Found 18042 images belonging to 13 classes.
{'battery': 0, 'biological': 1, 'brown-glass': 2, 'cardboard': 3, 'clothes': 4, 'glass': 5, 'green-glass': 6, 'metal': 7, 'paper': 8, 'plastic': 9, 'shoes': 10, 'trash': 11, 'white-glass': 12}


In [47]:
# Spliting our dataset into training, validation, and testing 
def split_dataset(source_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    random.seed(42)
    
    for class_folder in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_folder)
        if not os.path.isdir(class_path):
            continue 
        
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg','.jpeg','.png'))]
        random.shuffle(images)
    
        total = len(images)
        train_end = int(total * train_ratio)
        val_end = train_end + int(total * val_ratio)
    
        train_images = images[:train_end]
        val_images = images[train_end:val_end]
        test_images = images[val_end:]
    
        train_class_dir = os.path.join(output_dir, 'train', class_folder)
        val_class_dir = os.path.join(output_dir, 'val', class_folder)
        test_class_dir = os.path.join(output_dir,'test',class_folder)
    
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)
    
        for img in train_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(train_class_dir, img))

        for img in val_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(val_class_dir, img))

        for img in test_images:
            shutil.copy2(os.path.join(class_path, img), os.path.join(test_class_dir, img))

        print(f" {class_folder}: {len(train_images)} train / {len(val_images)} val / {len(test_images)} test")
    
if __name__ == "__main__":
    source = "../Datasets/combined-cleaned-dataset"   
    destination = "../Notebooks/the_final_sortdown"          
    split_dataset(source, destination, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)
    

 paper: 1150 train / 246 val / 248 test
 green-glass: 440 train / 94 val / 95 test
 clothes: 3727 train / 798 val / 800 test
 metal: 825 train / 176 val / 178 test
 cardboard: 905 train / 194 val / 195 test
 trash: 583 train / 125 val / 126 test
 glass: 350 train / 75 val / 76 test
 biological: 689 train / 147 val / 149 test
 white-glass: 542 train / 116 val / 117 test
 battery: 661 train / 141 val / 143 test
 brown-glass: 424 train / 91 val / 92 test
 plastic: 942 train / 202 val / 203 test
 shoes: 1383 train / 296 val / 298 test


In [48]:
# Data Augmentation
train_gen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    brightness_range = [0.8, 1.2]
)

print("Training Data:")
train_data = train_gen.flow_from_directory(
    "the_final_sortdown/train",
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=True
)


val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

print("\nValidation Data:")
val_data = val_gen.flow_from_directory(
    "the_final_sortdown/val",
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=True
)
print("\nTesting Data:")
test_data = test_gen.flow_from_directory(
    "the_final_sortdown/test",
    class_mode='categorical',
    target_size=(224, 224),
    batch_size=32,
    shuffle=False
)
class_names = list(test_data.class_indices.keys())



Training Data:
Found 16410 images belonging to 13 classes.

Validation Data:
Found 4985 images belonging to 13 classes.

Testing Data:
Found 5047 images belonging to 13 classes.


In [49]:
# Transfer Learning MobileNetV2
base_model = MobileNetV2(
    include_top = False,
    weights ='imagenet',
    input_shape = (224,224,3)
)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)

num_classes = len(train_data.class_indices)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

#model.summary()

In [50]:
early_stop = EarlyStopping(
    monitor='val_loss',   # What to monitor (validation loss is typical)
    patience=5,           # How many epochs without improvement before stopping
    restore_best_weights=True  # Optional, but restores the best model, not the last one
)


In [51]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs = 7, 
    callbacks = [early_stop]
)

/Users/cindychen/.pyenv/versions/3.9.16/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/7
513/513 ━━━━━━━━━━━━━━━━━━━━ 121s 233ms/step - accuracy: 0.7682 - loss: 0.7554 - val_accuracy: 0.9013 - val_loss: 0.3103
Epoch 2/7
220/513 ━━━━━━━━━━━━━━━━━━━━ 54s 186ms/step - accuracy: 0.8833 - loss: 0.3451

KeyboardInterrupt: 

In [8]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test accuarcy: {test_acc:.2f}")

/Users/cindychen/.pyenv/versions/3.9.16/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


158/158 ━━━━━━━━━━━━━━━━━━━━ 24s 153ms/step - accuracy: 0.9547 - loss: 0.1179
Test accuarcy: 0.94


In [10]:
# Adding Precision, Recall, and F1
from sklearn.metrics import classification_report

# Predict
y_pred = model.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get true labels
y_true = test_data.classes

# Print report
print(classification_report(y_true, y_pred_classes, target_names=class_names))


158/158 ━━━━━━━━━━━━━━━━━━━━ 25s 154ms/step
              precision    recall  f1-score   support

     battery       0.98      0.95      0.97       265
  biological       1.00      0.97      0.99       268
 brown-glass       0.84      0.90      0.87       173
   cardboard       0.94      0.99      0.96       360
     clothes       0.99      0.99      0.99      1493
       glass       0.44      0.26      0.33       140
 green-glass       0.83      0.92      0.87       175
       metal       0.90      0.97      0.93       333
       paper       0.99      0.93      0.96       470
     plastic       0.94      0.91      0.92       381
       shoes       0.97      0.99      0.98       545
       trash       0.94      0.91      0.93       235
 white-glass       0.72      0.80      0.76       209

    accuracy                           0.94      5047
   macro avg       0.88      0.89      0.88      5047
weighted avg       0.93      0.94      0.94      5047



In [3]:
# Adding Reinforcement Learning Sysytem
import gym 
from gym import spaces 
import numpy as np 

In [39]:
import gym
from gym import spaces
import numpy as np

class WasteDisposalEnv(gym.Env):
    def __init__(self, classifier_model, test_data, co2_impact_mapping, class_names, precomputed_probs=None):
        super(WasteDisposalEnv, self).__init__()
        self.classifier = classifier_model
        self.data = test_data
        self.co2_map = co2_impact_mapping  # e.g., {'plastic': 2.5, 'paper': 1.0, ...}
        self.class_names = class_names
        self.observation_space = spaces.Box(0, 1, shape=(224, 224, 3), dtype=np.float32)
        self.action_space = spaces.Discrete(len(class_names))
        self.index = 0
        self.images = self.data[0][0]  # sample batch
        self.labels = np.argmax(self.data[0][1], axis=1)
        self.precomputed_probs = precomputed_probs

    def reset(self):
        self.index = 0
        return self.images[self.index]

    def step(self, action):
        correct_label = self.labels[self.index]
        predicted_class = action
        co2_saved = 0
        done = False
        
        if predicted_class == correct_label:
            # Save CO2
            label_name = self.class_names[correct_label]
            co2_saved = self.co2_map.get(label_name, 0)
        else:
            # Penalty: use incorrect disposal CO2 emission (or 0 saving)
            co2_saved = -self.co2_map.get(self.class_names[correct_label], 0)

        self.index += 1
        if self.index >= len(self.images):
            done = True

        next_state = self.images[self.index % len(self.images)]
        return next_state, co2_saved, done, {}


In [33]:
co2_impact_mapping = {
    'battery': 10.0,
    'biological': 2.0,
    'brown-glass': 1.2,
    'cardboard': 1.0,
    'clothes': 4.0,
    'glass': 1.5,
    'green-glass': 1.3,
    'metal': 3.5,
    'paper': 1.0,
    'plastic': 2.5,
    'shoes': 3.0,
    'trash': -5.0,  # penalty if non-recyclable sent to trash
    'white-glass': 1.4
}


In [44]:
import numpy as np
# random agent, who does not recycle correctly 
env = WasteDisposalEnv(model, test_data, co2_impact_mapping, class_names)

total_reward = 0
obs = env.reset()
done = False

while not done:
    # Preprocess the observation (if not already normalized/resized)
    input_image = np.expand_dims(obs, axis=0)  # add batch dimension
    prediction = model.predict(input_image)
    action = env.action_space.sample()

    obs, reward, done, info = env.step(action)
    total_reward += reward

print(f"Total CO₂ saved: {total_reward:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

In [63]:
import numpy as np
# smart agent, who does recycle correctly 
env = WasteDisposalEnv(model, test_data, co2_impact_mapping, class_names)

total_reward = 0
obs = env.reset()
done = False

while not done:
    # Preprocess the observation (if not already normalized/resized)
    input_image = np.expand_dims(obs, axis=0)  # add batch dimension
    prediction = model.predict(input_image)
    action = np.argmax(prediction)

    obs, reward, done, info = env.step(action)
    total_reward += reward

print(f"Total CO₂ saved: {total_reward:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

In [65]:
import numpy as np
import random  # needed for random.random()

env = WasteDisposalEnv(model, test_data, co2_impact_mapping, class_names)

total_reward = 0
obs = env.reset()
done = False

epsilon = 0.1  # 10% exploration

while not done:
    if random.random() < epsilon:

        action = env.action_space.sample()
    else:
    
        input_image = np.expand_dims(obs, axis=0)
        prediction = model.predict(input_image, verbose=0)
        action = np.argmax(prediction)

    obs, reward, done, info = env.step(action)
    total_reward += reward

print(f"Total CO₂ saved: {total_reward:.2f}")


Total CO₂ saved: 240.00


In [4]:
class WasteDisposalEnv(gym.Env): 

    def __init__(self, classifier_model, test_data, co2_impact_mapping, class_names, precomputed_probs=None):

        super(WasteDisposalEnv, self).__init__()

        #used for predicting material type 

        self.classifier = classifier_model 

        #data for predict 

        self.data = test_data
        self.class_names = class_names
        self.images, self.labels = self._prepare_data()  
        self.co2_impact_mapping = co2_impact_mapping 
        
        if precomputed_probs is not None:
            self.predictions = precomputed_probs
            self.images, self.labels = self._prepare_data_labels_only()  # only get labels

        else:
            self.images, self.labels = self._prepare_data()  
            self.predictions = self.classifier.predict(self.images, verbose=1)

    
        #define action space 4 disposal options(Recycle, Compost, Donate, Landfill)
        self.action_space = spaces.Discrete(4)
        self.predictions = precomputed_probs
        self.observation_space = spaces.Box(low=0, high=1, shape=(self.classifier.output_shape[-1],), dtype=np.float32)
        
        self.current_idx = 0


    def _prepare_data(self):
        images = []
        labels = []
        for batch_x, batch_y in self.data:
            images.append(batch_x)
            labels.append(np.argmax(batch_y, axis=1))
        return np.concatenate(images), np.concatenate(labels)

        
    def reset(self): 
        self.current_idx = 0 
        # image = self.images[self.current_idx]
        # probs = self.classifier.predict(image[np.newaxis, ...])[0]
        return self.predictions[self.current_idx]
    
    def step(self, action):
        image = self.images[self.current_idx]
        true_label = self.labels[self.current_idx]

        probs = self.predictions[self.current_idx]
        predicted_class = np.argmax(probs)

        reward = self._calculate_reward(predicted_class, action)

        self.current_idx += 1
        done = self.current_idx >= len(self.images)

        if not done:
            next_image = self.images[self.current_idx]
            next_probs = self.classifier.predict(next_image[np.newaxis, ...])[0]
        else:
            next_probs = np.zeros_like(probs)

        return next_probs, reward, done, {}
        
    def _calculate_reward(self, predicted_class, action):
        item_type = self.class_names[predicted_class]
        impact_info = self.co2_impact_mapping.get(item_type, None)

        if impact_info is None:
            return -1  # Unknown item, small penalty

        max_co2_saved = impact_info["max_co2_saved"]
        actions = impact_info["actions"]

        action_name = {0: "recycle", 1: "compost", 2: "donate", 3: "landfill"}.get(action, None)

        if action_name is None:
            return -1  # Invalid action penalty

        # Get multiplier for action
        action_multiplier = actions.get(action_name, 0.0)  # 0 if action not listed

        # Reward is proportional
        reward = max_co2_saved * action_multiplier
        return reward


In [5]:
co2_impact_mapping = {
    "plastic": {
        "max_co2_saved": 10,  # in kg CO2
        "actions": {
            "recycle": 1.0,    # 100% of CO₂ savings if recycled
            "landfill": 0.0    # 0% savings if landfilled
        }
    },
    "food_waste": {
        "max_co2_saved": 8,
        "actions": {
            "compost": 1.0,
            "landfill": 0.1    # 10% saving (maybe still some methane capture)
        }
    },
    "electronics": {
        "max_co2_saved": 15,
        "actions": {
            "donate": 1.0,
            "landfill": 0.0
        }
    },
    "textile": {
        "max_co2_saved": 12,
        "actions": {
            "donate": 1.0,
            "landfill": 0.2    # Landfilling might still recover some energy
        }
    }
}


In [ ]:
class_names = list(test_data.class_indices.keys())
all_probs = model.predict(test_data, verbose=1)

env = WasteDisposalEnv(classifier_model=model, 
                       test_data=test_data, 
                       co2_impact_mapping=co2_impact_mapping, 
                       class_names=class_names,
                       precomputed_probs=all_probs)

episode_rewards = []
co2_savings = []

num_episodes = 100  # you can change this to 100 later if you want
all_probs = model.predict(test_data, verbose=1)

for episode in range(num_episodes):
    observation = env.reset()
    done = False
    total_reward = 0
    total_co2_saved = 0
     
    while not done:
        # Random action for now (you can replace with a policy later)
        action = env.action_space.sample()

        next_observation, reward, done, info = env.step(action)

        total_reward += reward
        total_co2_saved += reward  # because reward is proportional to CO₂ saved

        observation = next_observation

    episode_rewards.append(total_reward)
    co2_savings.append(total_co2_saved)


In [ ]:
print(f"Episode {episode}: Total Reward = {total_reward}, Total CO₂ Saved = {total_co2_saved:.2f} kg")

In [ ]:
import matplotlib.pyplot as plt

# Plot total reward (CO₂ saved) per episode
plt.figure(figsize=(10, 6))
plt.plot(episode_rewards, marker='o')
plt.title('Total CO₂ Saved per Episode')
plt.xlabel('Episode')
plt.ylabel('Total CO₂ Saved (kg)')
plt.grid(True)
plt.show()
